全连接网络与卷积神经网络
===

# 1.全连接网络
全连接神经网络之所以不太适合图像识别任务，主要有以下几个方面的问题：
1. 参数数量太多:考虑一个输入1000*1000像素的图片(一百万像素，现在已经不能算大图了)，输入层有1000*1000=100万节点。假设第一个隐藏层有100个节点(这个数量并不多)，那么仅这一层就有(1000*1000+1)*100=1亿参数，这实在是太多了！我们看到图像只扩大一点，参数数量就会多很多，因此它的扩展性很差。
2. 没有利用像素之间的位置信息:对于图像识别任务来说，每个像素和其周围像素的联系是比较紧密的，和离得很远的像素的联系可能就很小了。如果一个神经元和上一层所有神经元相连，那么就相当于对于一个像素来说，把图像的所有像素都等同看待，这不符合前面的假设。当我们完成每个连接权重的学习之后，最终可能会发现，有大量的权重，它们的值都是很小的(也就是这些连接其实无关紧要)。努力学习大量并不重要的权重，这样的学习必将是非常低效的。
3. 网络层数限制:我们知道网络层数越多其表达能力越强，但是通过梯度下降方法训练深度全连接神经网络很困难，因为全连接神经网络的梯度很难传递超过3层。因此，我们不可能得到一个很深的全连接神经网络，也就限制了它的能力。

# 2.卷积神经网络的层级结构
数据层、卷积层、激励层、池化层(pooling)、全连接层(FC-Full Connect)。层次是穿插的，每一层都有一个简单的API：用一些含有或者不含有参数的可导函数构成，将输入的3D数据变为3D的输出数据.一个简单的卷积神经网络是有各种层按照顺序排列而成，网络中的每个层使用一个可以微分的函数将激活数据从一个层传递到另一个层<br/>
举个例子：
1. 数据层输入$[32*32*3]$的存有图像的原始像素值
2. 卷积层中，神经元与输入层中的一个局部区域相连，每个神经元都计算自己与输入层相连的小区域与自己权重的内积。卷积曾会计算所有神经元的输出。如果我们使用12个滤波器(卷积核)，得到的输出数据体的维度就是$[32*32*12]$
3. 汇聚层(Pooling)在空间维度上进行降采样(downsampling)操作，数据尺寸变为$[16*16*12]$。
4. 全连接层将会计算分类评分，数据尺寸变味$[1*1*10]$，其中10个数字对应的就是10个类别的分类评分值

## 2.1.数据层
数据过来，需要在这一层进行一些数据预处理的操作，常见的3种数据预处理的方法有

### 2.1.1.去均值 Mean subtraction
把输入数据各个维度都中心化到0(只是针对训练集！！！)，首先计算训练集的均值，然后把每一个图片都减去这个均值(测试集上也是减这个均值，不要去计算测试集上的均值)，主要原因是光照会影响.去均值也叫做均值减法，是晕处理最常用的形式。它对数据中每个独立特征减去平均值，从几何上可以理解为在每个维度上都将数据云的中心迁移到原点。在numpy中，该操作可以通过代码$X -= np.mean(X, axis=0)$实现。而对于图像，更常用的是对所有像素都减去一个值，可以用$X -= np.mean(X)$实现，也可以在3个颜色通道上分别操作

### 2.1.2.归一化Normalization
幅度归一化到一定的范围(CNN操作一般不做这个操作，因为RGB就是在0~255这个范围之内的)
1. 方法一：先对数据做零中心化(zero-centered)处理，然后每个维度都除以其标准差，$X /= np.std(X, axis=0)$
2. 方法二：对每个维度都做归一化，使得每个维度的最大和最小是1和-1

### 2.1.3.PCA/白化 Whitening
用PCA降维在这种处理中，先对数据进行零中心化处理，然后计算协方差矩阵，它展示了数据中的相关性结构$U,S,V=np.linalg.svd(cov)$。白化是对数据每个特征轴上的幅度归一化(CNN一般也不用)。白化操作的输入是特征基准上的数据，然后对每个维度除以其特征值来对数据范围进行归一化，几何解释是，如果数据服从多变量的高斯分布，经过白化后，数据的分布将会是一个均值为0，且协方差相等的矩阵。$Xwhite = \frac{Xrot}{np.sqrt(S + 1e-5)}$。<br/>
左边是原始数据，中间是做了PCA之后的数据，相当于先零中心化，然后做了旋转(去相关性)，右边是做了白化之后的操作

## 2.2.卷积层
卷积神经网络解决问题的三个思路：
- 局部连接 这个是最容易想到的，每个神经元不再和上一层的所有神经元相连，而只和一小部分神经元相连。这样就减少了很多参数。
- 权值共享 一组连接可以共享同一个权重，而不是每个连接有一个不同的权重，这样又减少了很多参数。
- 下采样 可以使用Pooling来减少每层的样本数，进一步减少参数数量，同时还可以提升模型的鲁棒性。

对于图像识别任务来说，卷积神经网络通过尽可能保留重要的参数，去掉大量不重要的参数，来达到更好的学习效果。

### 2.2.1.构成
卷积神经网络的基本结构是有多个卷积核组合形成的，每个卷积核同输入数据卷积运算，形成新的特征"图"

### 2.2.2.卷积层的关键参数
- 奇偶选择：一般为奇数，满足对称
- 大小选择：根据输入数据，根据图像特征
- 厚度：与输入数据一致
- 覆盖范围：一般覆盖全部输入，特殊情况覆盖局部区域
- 步长:一般情况下，计算都是步长为1的，得到的结果就是$3*3$，相当于做了pooling
- 边界扩充:在卷积计算过程中，为了允许边界上的数据也能作为中心参与卷积计算，将边界假装延伸，它的目的就是确保卷积后特征图尺寸一致，如果卷积核的宽度是$2i+1$，那么pad宽度就是i

### 2.2.3.总结
- 同输入数据进行计算的二维(一维(语音识别)、三维)算子
- 大小用户定义，深度-输入数据定义
- 卷积核"矩阵"值：卷积神经网络的参数
- 卷积核初值随机生成，通过反向传播更新
- 卷积核是卷积组成单位
- 卷积核数目，卷积计算的步长，pad对卷积层输出的影响
- 卷积层参数数目计算
- 正向，反向卷积运算表达式

反向求导计算中，卷积核的参数跟上一层的导数以及本层的输入有关，每一层的$\Delta{x},\Delta{y}$和上一层的梯度和本层的卷积核参数有关.卷积后的尺寸跟卷积前的尺寸的计算公式：
$$\begin{eqnarray}
W_2=(W_1-F+2P)/S+1\\
H_2=(H_1-F+2P)/S+1
\end{eqnarray}$$
其中$W_1,H_1$是卷积前图片的宽度和高度，$W_2,H_2$是卷积后图片的宽度和高度，$F$是卷积核的尺寸，$P$是padding的圈数，补几圈0就是几，$S$是步长

## 2.3.激励层 ReLU
把卷积层输出结果做非线性映射
- 不要用Sigmoid
- 首先使用ReLU，因为它快，但是要小心，需要打印中间状态
- 如果ReLU失效，使用Leaky ReLU或者Maxout
- 某些情况下tanh有不错的效果，但是很少

## 2.4.池化层 Pooling layer
数据进行压缩，夹在连续的卷积层中间，压缩数据和参数的量，减小过拟合。它可以进行特征融合，可以尺寸降维，不是特征降维。具体方法有
- 最大化池化
- 平均池化

## 2.5.全连接层
作用是推力器、分类器，它等于全局卷积。

## 2.6.Softmax-归一化层
将结果压缩至(0,1)，所有元素和为1，所用的公式就是$\sigma(z)_j=\frac{e^{z_j}}{\sum_{k=1}^Ke^{z_k}}, j=1,...,K$